In [2]:
%pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 5.0 MB/s eta 0:00:00


In [3]:
import gensim
from gensim.models import Word2Vec
import re
import pymorphy3
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from nltk.corpus import stopwords
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
# from my_parsers import download, parse
from tqdm import tqdm
from collections import Counter
# from brown_clustering import BigramCorpus, BrownClustering

pd.options.display.max_rows = 200
tqdm.pandas()

ModuleNotFoundError: No module named 'catboost'

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
# Инициализация анализатора pymorphy2 для лемматизации
morph = pymorphy3.MorphAnalyzer()

# Функция для лемматизации слова
def lemmatize_word(word):
    return morph.parse(word)[0].normal_form

# Функция для предобработки данных (разбиение на слова, лемматизация, удаление пунктуации)
def preprocess_names(company_names):
    processed_names = []
    for name in tqdm(company_names):
        # Приводим название к нижнему регистру
        name = name.lower()
        # Удаляем всю пунктуацию, оставляем только слова
        name = re.sub(r'[^\w\s]', '', name)
        # Разбиваем на отдельные слова
        words = name.split()

        # Лемматизируем каждое слово
        # lemmatized_words = [lemmatize_word(word) for word in words]

        processed_names.append(words)
    return processed_names


def get_emb_by_modele(model, comp_names_without_job, column_prefix):
    all_tokens = set(model.wv.index_to_key)
    word_vectors_dict = {word: model.wv[word] for word in model.wv.index_to_key}

    all_embds = []

    for sent in tqdm(comp_names_without_job):
        all_emb = [word_vectors_dict[word] for word in sent if word in all_tokens]

        if len(all_emb) == 0:
            emb = np.zeros(100)
        else:
            emb = np.mean(all_emb, axis=0)

        all_embds.append(emb)
    emb_comp_name_df = pd.DataFrame(np.array(all_embds), columns=[f"{column_prefix}_{i}" for i in range(100)])
    return emb_comp_name_df

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_emb_by_tfidf(processed_sentences, column_prefix, max_features=100, ):
    tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
    tfidf_matrix = tfidf_vectorizer.fit_transform(processed_sentences)

    # Преобразуем разреженную матрицу TF-IDF в плотный формат и создаем DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f"{column_prefix}_{word}" for word in tfidf_vectorizer.get_feature_names_out()])
    return tfidf_vectorizer, tfidf_df

In [ ]:
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

In [ ]:
sentences = preprocess_names(company_names)

In [ ]:
model_demand = Word2Vec(sentences, vector_size=100, window=8, min_count=1, sg=1)
# Сохранение обученной модели
model_demand.save("demand_word2vec_russian.model")

In [ ]:
comp_names_without_job = preprocess_names(df_TRAIN_RES_1["company_name"])
emb_comp_name_df = get_emb_by_modele(model_comp_name, comp_names_without_job, column_prefix="comp_name_emb")
df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True), emb_comp_name_df.reset_index(drop=True)], axis=1)

In [ ]:
sentences = preprocess_names(df_TRAIN_RES_1["demands"])

RETRAIN_MODEL = True
column_prefix = "demnds_tfidf"

if RETRAIN_MODEL:
    demands_vectorizer, demands_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                     column_prefix=column_prefix)

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            demands_tfidf.reset_index(drop=True)], axis=1)